# SQL Injection

### The purpose of this exercise is to demonstrate how unsanitized database queries can lead to leaks of personal information. 

### In this exercise we will:

<ul>
<li>Connect to a database.</li>
<li>Submit a legitimate query.</li>
<li>Submit a variety of illicit queries.</li>
<li>Demonstrate how to sanitize database inputs to reduce the occurance of SQL injection attacks.</li>
</ul>

A sample SQLite database has been created that stores information about a set of employees, including their names, social security numbers, home address, and other information.  The follow code snippet demonstrates how to connect to the database and query that infomration using the SQL _SELECT_ statement.

In [22]:
from background import *

connection = create_connection(".\\test.db")

select_users = "SELECT * from EMPLOYEE"
users = execute_read_query(connection, select_users)

for user in users:
    print(user) 

Connection to SQLite DB successful


There are situations where one may not wish a user to be able to see information about arbitrary entires in the database.  For instance, the next code snippet requires a user to enter their SSN in order to see their current information.  The assumption here is that one only knows their own SSN, and not the SSN of their co-workers.  Try the example for the employee whose SSN is 123456789.

In [23]:
ssn="123456789"

In [24]:
select_users = "SELECT * from employee where ssn=" + ssn

users = execute_read_query(connection, select_users)

for user in users:
    print(user) 

Very good.  Now a user can only see information for SSN numbers they know.  There is a first, obvious, vulerability in that a user could guess random SSNs and, perhaps, leak information about their co-workers, but the search space is large.  A second, more insidious vulnerable is known as _SQL Injection_ where additional SQL code can be fed into the application to change the results.  Append "OR TRUE" to the SSN and run the sample again to see what happens.

As you can see, because of the mechanism the code uses to build their query, it is vulnerable to manipulation of the where clause.  Let's run a new version that _sanitizes_ the user input to ensure it only contains a valid SSN.

<img src="https://imgs.xkcd.com/comics/exploits_of_a_mom.png">

In [1]:
if(isSSN(ssn)):
    select_users = "SELECT * from employee where ssn=" + ssn

    users = execute_read_query(connection, select_users)

    for user in users:
        print(user) 

NameError: name 'isSSN' is not defined

In summary, if you construct a dynamic SQL query based on user input make sure to sanitize your inputs!